In [1]:
from dotenv import load_dotenv
import os
#import json

In [2]:
from util import run_command

In [3]:
# Load environment variables from the .env file
load_dotenv()

# Access environment variables
TUNER_ID = os.getenv('TUNER_ID')
TUNER_IP = os.getenv('TUNER_IP')

TUNER_ID, TUNER_IP

('10B16BC7', '192.168.0.151')

In [4]:
RECORDINGS = os.getenv('DVR_RECORDINGS')
RECORDINGS

'/home/tony/Videos'

In [5]:
cmd=f'echo curl -o {RECORDINGS}/tony1nbc.ts {TUNER_IP}:5004/auto/v46.1?duration=3600 | at 13:25'


In [6]:
cmd

'echo curl -o /home/tony/Videos/tony1nbc.ts 192.168.0.151:5004/auto/v46.1?duration=3600 | at 13:25'

In [7]:
#out, stat = run_command(cmd)

In [16]:
cmd=f'echo curl -o {RECORDINGS}/golf.ts {TUNER_IP}:5004/auto/v46.1?duration=10800 | at -t 202501111500'
out, stat = run_command(cmd)
stat, out

(0, '')

In [17]:
! atq

3	Sat Jan 11 15:00:00 2025 a tony


In [18]:
! date

Sat Jan 11 02:33:44 PM CST 2025


In [19]:
! atrm 2

Cannot find jobid 2


In [21]:
! at -c 3

#!/bin/sh
# atrun uid=1000 gid=1000
# mail tony 0
umask 2
LESSOPEN=\|\ /usr/bin/lesspipe\ %s; export LESSOPEN
CONDA_PROMPT_MODIFIER=\(tv\)\ ; export CONDA_PROMPT_MODIFIER
TUNER_IP=192.168.0.151; export TUNER_IP
LANGUAGE=en_US; export LANGUAGE
USER=tony; export USER
XDG_SEAT=seat0; export XDG_SEAT
MPLBACKEND=module://matplotlib_inline.backend_inline; export MPLBACKEND
XDG_SESSION_TYPE=x11; export XDG_SESSION_TYPE
SHLVL=1; export SHLVL
OLDPWD=/home/tony/opt/dvr1/8_study; export OLDPWD
CONDA_SHLVL=2; export CONDA_SHLVL
HOME=/home/tony; export HOME
DESKTOP_SESSION=cinnamon; export DESKTOP_SESSION
XDG_SEAT_PATH=/org/freedesktop/DisplayManager/Seat0; export XDG_SEAT_PATH
GTK_MODULES=gail:atk-bridge; export GTK_MODULES
PAGER=cat; export PAGER
MAKEFLAGS=; export MAKEFLAGS
DBUS_SESSION_BUS_ADDRESS=unix:path=/run/user/1000/bus; export DBUS_SESSION_BUS_ADDRESS
COLORTERM=truecolor; export COLORTERM
_CE_M=; export _CE_M
GNOME_KEYRING_CONTROL=/run/user/1000/keyring; export GNOME_KEYRING_CONTROL
MAKE